### EWMA and Constant Volatility forecast

For the Exponentially Weighted Moving Average (EWMA), we used a $\lambda=.94$ and a 252 days window (~1 year)

In [42]:
import sys
!{sys.executable} -m pip install numpy pandas numpy openpyxl | grep -v 'already satisfied'

import pandas as pd
import numpy as np
import openpyxl

In [126]:
df = pd.read_excel("Data.xlsx", sheet_name="df_sp500")

df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
df.columns = ["Date", "Open", "High", "Low","Close", "Volume"]
df["Returns"]=df['Close'].pct_change()*100

df.dropna(inplace=True)
df["vol"] = df["Returns"].abs()

df.head()

,Date,Open,High,Low,Close,Volume,Returns,vol
1,2018-06-12,2785.600098,2789.800049,2778.780029,2786.850098,3435700000,0.174339,0.174339
2,2018-06-13,2787.939941,2791.469971,2774.649902,2775.629883,3820770000,-0.402613,0.402613
3,2018-06-14,2783.209961,2789.060059,2776.520020,2782.489990,3540930000,0.247155,0.247155
4,2018-06-15,2777.780029,2782.810059,2761.729980,2779.659912,5451990000,-0.101710,0.101710
5,2018-06-18,2765.790039,2774.989990,2757.120117,2773.750000,3316120000,-0.212613,0.212613


In [127]:
returns = pd.Series(df["Returns"])

lambda_val = 0.94
alpha_val = 1 - lambda_val

volatility = returns.ewm(alpha=alpha_val, min_periods=252).std()

In [128]:
volatility.describe()

count    1007.000000
mean        1.237271
std         0.773205
min         0.433402
25%         0.785170
50%         1.059414
75%         1.417645
max         5.322654
Name: Returns, dtype: float64

In [129]:
df["vol"].describe()

count    1258.000000
mean        0.894456
std         1.036102
min         0.000687
25%         0.251420
50%         0.630031
75%         1.165873
max        11.984055
Name: vol, dtype: float64

In [130]:
df.head()

,Date,Open,High,Low,Close,Volume,Returns,vol
1,2018-06-12,2785.600098,2789.800049,2778.780029,2786.850098,3435700000,0.174339,0.174339
2,2018-06-13,2787.939941,2791.469971,2774.649902,2775.629883,3820770000,-0.402613,0.402613
3,2018-06-14,2783.209961,2789.060059,2776.520020,2782.489990,3540930000,0.247155,0.247155
4,2018-06-15,2777.780029,2782.810059,2761.729980,2779.659912,5451990000,-0.101710,0.101710
5,2018-06-18,2765.790039,2774.989990,2757.120117,2773.750000,3316120000,-0.212613,0.212613


In [131]:
volatility = pd.concat([pd.Series([np.nan]), volatility], ignore_index=True)
volatility = volatility[:-1]

In [132]:
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
df["ewma"]=volatility

In [133]:
MA_volatility = returns.rolling(window=252).std()
MA_volatility = pd.concat([pd.Series([np.nan]), MA_volatility], ignore_index=True)
MA_volatility = MA_volatility[:-1]
df["MA"]=MA_volatility
df

,Date,Open,High,Low,Close,Volume,Returns,vol,ewma,MA
0,2018-06-12,2785.600098,2789.800049,2778.780029,2786.850098,3435700000,0.174339,0.174339,NaN,NaN
1,2018-06-13,2787.939941,2791.469971,2774.649902,2775.629883,3820770000,-0.402613,0.402613,NaN,NaN
2,2018-06-14,2783.209961,2789.060059,2776.520020,2782.489990,3540930000,0.247155,0.247155,NaN,NaN
3,2018-06-15,2777.780029,2782.810059,2761.729980,2779.659912,5451990000,-0.101710,0.101710,NaN,NaN
4,2018-06-18,2765.790039,2774.989990,2757.120117,2773.750000,3316120000,-0.212613,0.212613,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1253,2023-06-05,4282.990234,4299.279785,4266.819824,4273.790039,3813290000,-0.200358,0.200358,0.868888,1.320570
1254,2023-06-06,4271.339844,4288.330078,4263.089844,4283.850098,3996560000,0.235390,0.235390,0.848079,1.315632
1255,2023-06-07,4285.470215,4299.189941,4263.959961,4267.520020,4537800000,-0.381201,0.381201,0.822348,1.311541
1256,2023-06-08,4268.689941,4298.009766,4261.069824,4293.930176,3826740000,0.618864,0.618864,0.808875,1.311663


In [135]:
cols = ["Date", "Close", "Returns", "vol", "ewma", "MA"]
Final_data = df[cols]
Final_data

,Date,Close,Returns,vol,ewma,MA
0,2018-06-12,2786.850098,0.174339,0.174339,NaN,NaN
1,2018-06-13,2775.629883,-0.402613,0.402613,NaN,NaN
2,2018-06-14,2782.489990,0.247155,0.247155,NaN,NaN
3,2018-06-15,2779.659912,-0.101710,0.101710,NaN,NaN
4,2018-06-18,2773.750000,-0.212613,0.212613,NaN,NaN
...,...,...,...,...,...,...
1253,2023-06-05,4273.790039,-0.200358,0.200358,0.868888,1.320570
1254,2023-06-06,4283.850098,0.235390,0.235390,0.848079,1.315632
1255,2023-06-07,4267.520020,-0.381201,0.381201,0.822348,1.311541
1256,2023-06-08,4293.930176,0.618864,0.618864,0.808875,1.311663


In [136]:
Final_data.to_csv('EWMA_MA.csv', index=False)